## Quadratization of the system

---

[[Package page]](https://github.com/AndreyBychkov/QBee/)

QBee is a Python library for transforming systems of differential equations into a systems with quadratic right-rand side. 

Here is a online program version of the packege: [link](https://huggingface.co/spaces/Armaliltril/qbee)

In [1]:
import sympy
from qbee import *

sympy.init_printing()  # If you work in Jupyter notebook 

$$
\left\{\begin{array}{l}
c_1^{\prime}=-A \exp \left(-E_a /\left(R_u T\right)\right) c_1^{0.2} c_2^{1.3} \\
c_2^{\prime}=2 c_1^{\prime} \\
c_3^{\prime}=-c_1^{\prime} \\
c_4^{\prime}=-2 c_1^{\prime}
\end{array}\right.
$$

In [4]:
A, Ea, Ru = parameters("A, Ea, Ru")
c1, c2, c3, c4, T = functions("c1, c2, c3, c4, T")  

eq1 = -A * sp.exp(-Ea / (Ru * T)) * c1 ** 0.2 * c2 ** 1.3
system = [
    (c1, eq1),
    (c2, 2 * eq1),
    (c3, -eq1),
    (c4, -2 * eq1)
]

# {T: 2} means than T can have a derivative of order at most two => T''
quadr_system = polynomialize_and_quadratize(system, input_der_orders={T: 2})
if quadr_system:
    print("Quadratized system:")
    print(quadr_system)

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:
w_{0} = c1**(-0.8)
w_{1} = c2**(-0.7)
w_{2} = 1/T
w_{3} = exp(-Ea*w_{2}/Ru)


Nodes processed: 80 nodes [00:00, 96.37 nodes/s, Current best order = 5]  



Elapsed time: 0.867s.


Nodes processed: 117 nodes [00:00, 184.19 nodes/s, Current best order = 5]



Elapsed time: 0.645s.
Quadratization result
Number of introduced variables: 5
Nodes traversed: 117
Introduced variables:
w_{4} = T'*w_{2}
w_{5} = T'*w_{2}**2
w_{6} = w_{2}**2
w_{7} = c2**2*w_{0}*w_{1}*w_{3}
w_{8} = c1*c2*w_{0}*w_{1}*w_{3}

Quadratized system:
c1' = -A*c2*w_{8}
c2' = -2*A*c2*w_{8}
c3' = A*c2*w_{8}
c4' = 2*A*c2*w_{8}
w_{0}' = 4*A*w_{0}*w_{7}/5
w_{1}' = 7*A*w_{1}*w_{8}/5
w_{2}' = -T'*w_{6}
w_{3}' = Ea*w_{3}*w_{5}/Ru
T' = T'
T'' = T''
T''' = 0
w_{4}' = -T'*w_{5} + T''*w_{2}
w_{5}' = T''*w_{6} - 2*w_{4}*w_{5}
w_{6}' = -2*w_{4}*w_{6}
w_{7}' = 4*A*w_{7}**2/5 - 13*A*w_{7}*w_{8}/5 + Ea*w_{5}*w_{7}/Ru
w_{8}' = -A*w_{7}*w_{8}/5 - 3*A*w_{8}**2/5 + Ea*w_{5}*w_{8}/Ru


Here are some ODE systems that we try to quadratize:
- $x'=-x+ax^{3}$
- $x'=-x^3$
- $x'' = kx+ax^{3}$ 

  (we set $x_{0}=x$ and $x_{1}=x'$)

  $x_{0}'=x_1$

  $x_{1}'=-kx_{0}-ax_{0}^{3}$
  

### System 1
$$
x'=-x+ax^{3}
$$
Quadratization:
$$
y' = -2y+2ay^{2}
$$
where $y=x^2$

In [2]:
a = parameters("a")
x = functions("x")

eq = - x + a * x ** 3
system = [(x, eq)]

quadr_system = polynomialize_and_quadratize(system, 
                                            new_vars_name = "w")

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Nodes processed: 4 nodes [00:00, 30.56 nodes/s, Current best order = 1]   




Elapsed time: 0.056s.


Nodes processed: 3 nodes [00:00, 101.32 nodes/s, Current best order = 1]



Elapsed time: 0.051s.
Quadratization result
Number of introduced variables: 1
Nodes traversed: 3
Introduced variables:
w{0} = x**2



In [19]:
from sympy import exp

x, y, u = functions("x, y, u")
p = parameters("p")
quad_res = polynomialize_and_quadratize([(x, y / (1 + exp(-p * x))),
                                         (y, x * exp(y) + u)],
                                        input_der_orders={u: 0},
                                        new_vars_name='z',
                                        start_new_vars_with=1)
print(quad_res)

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:
z{1} = exp(-p*x)
z{2} = 1/(z{1} + 1)
z{3} = exp(y)


Nodes processed: 22 nodes [00:00, 466.54 nodes/s, Current best order = 4]  



Elapsed time: 0.053s.


Nodes processed: 44 nodes [00:00, 147.06 nodes/s, Current best order = 4]


Elapsed time: 0.321s.
Quadratization result
Number of introduced variables: 4
Nodes traversed: 44
Introduced variables:
z{4} = x*z{3}
z{5} = y*z{2}
z{6} = y*z{1}*z{2}**2
z{7} = z{1}*z{2}**2

x' = z{5}
y' = u + z{4}
z{1}' = -p*z{1}*z{5}
z{2}' = p*z{5}*z{7}
z{3}' = u*z{3} + z{3}*z{4}
u' = 0
z{4}' = u*z{4} + z{3}*z{5} + z{4}**2
z{5}' = p*z{5}*z{6} + u*z{2} + z{2}*z{4}
z{6}' = -p*z{5}*z{6} + 2*p*z{6}**2 + u*z{7} + z{4}*z{7}
z{7}' = -p*z{5}*z{7} + 2*p*z{6}*z{7}


### System 2
$$
x'=-x^{3}
$$

In [3]:
x = functions("x")

eq = - x ** 3
system = [(x, eq)]

quadr_system = polynomialize_and_quadratize(system, 
                                            new_vars_name = "w")

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Nodes processed: 4 nodes [00:00, 184.78 nodes/s, Current best order = 1]  




Elapsed time: 0.009s.


Nodes processed: 3 nodes [00:00, 555.54 nodes/s, Current best order = 1] 


Elapsed time: 0.009s.
Quadratization result
Number of introduced variables: 1
Nodes traversed: 3
Introduced variables:
w{0} = x**2



### System 3
$$
x'' = kx+ax^{3}
$$
(we set $x_{0}=x$ and $x_{1}=x'$). Then we have the system of the ODEs:
$$
\begin{cases}
x_{0}'=x_1\\
x_{1}'=kx_{0}+ax_{0}^{3}
\end{cases}

In [4]:
k, a = parameters("k, a")
x0, x1 = functions("x0, x1")

eq1 = -k * x0 - a * x0**3
system = [
    (x1, eq1),
    (x0, x1),
]

quadr_system = polynomialize_and_quadratize(system, input_der_orders={x0: 2})

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Nodes processed: 2 nodes [00:00, 16.08 nodes/s, Current best order = inf] 

Nodes processed: 121 nodes [00:00, 126.47 nodes/s, Current best order = 4]



Elapsed time: 0.834s.


Nodes processed: 83 nodes [00:00, 140.98 nodes/s, Current best order = 4]


Elapsed time: 0.598s.
Quadratization result
Number of introduced variables: 4
Nodes traversed: 83
Introduced variables:
w_{0} = x0**3
w_{1} = x0**3
w_{2} = x0*x0'
w_{3} = x0**2

